
# Media Mix Modeling (MMM) — Python Notebook

**When to Use**  
- You need **channel-level impact** estimates using **aggregate** (weekly/monthly) spend + outcome (sales/revenue) data.  
- You’re allocating **budgets across channels** (TV, Paid Social, Search, OOH, etc.) and user-level tracking is incomplete or deprecated.

**Best Application**  
- Strategic planning and **budget reallocation** across channels.  
- Estimating **diminishing returns** (saturation) and **carryover** (adstock).  
- Running **what-if** scenarios on spend changes.

**When Not to Use**  
- **Short-term tactical** optimization (creative/keyword bidding); use MTA/experimentation instead.  
- Highly **non-stationary** environments with rapid structural breaks and minimal history.  
- When you must attribute at the **user or creative** level.

**How to Interpret Results**  
- **Coefficients** reflect the relationship between *transformed* media (adstock + saturation) and outcome.  
- **Channel contributions** estimate each channel’s share of modeled sales.  
- **Marginal ROI (mROI)** indicates which channel yields the **biggest incremental return** for the next dollar of spend (at current levels).  
- Use results for **budget shifts**, not as immutable truth—validate with **experiments/geo tests**.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import RidgeCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)
plt.rcParams['figure.figsize'] = (8,4)


In [ ]:

from io import StringIO

csv_data = StringIO("""
week,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9
6,8.7,48.9,75,7.2
7,57.5,32.8,23.5,11.8
8,120.2,19.6,11.6,13.2
9,8.6,2.1,1,4.8
10,199.8,2.6,21.2,10.6
11,66.1,5.8,24.2,8.6
12,214.7,24,4,17.4
13,23.8,35.1,65.9,9.2
14,97.5,7.6,7.2,9.7
15,204.1,32.9,46,19
16,195.4,47.7,52.9,22.4
17,67.8,36.6,114,12.5
18,281.4,39.6,55.8,24.4
19,69.2,20.5,18.3,11.3
20,147.3,23.9,19.1,14.6
21,218.4,27.5,25.0,18.0
22,237.4,5.1,23.5,12.5
23,13.2,15.9,49.6,5.6
24,228.3,16.9,26.2,15.5
25,62.3,12.6,18.3,9.7
26,262.9,3.5,19.5,12.0
27,142.9,29.3,12.6,15.0
28,240.1,16.7,22.9,16.7
29,248.8,27.1,22.9,18.9
30,70.6,16.0,40.8,10.5
""")
df = pd.read_csv(csv_data)
df = df.sort_values('week').reset_index(drop=True)

# Example seasonal effect: every 4th week high (toy illustration)
df['seasonality'] = ((df['week'] % 4)==0).astype(int)

df.head()


### Adstock & Saturation Transforms

In [ ]:

def adstock(x, rate=0.5):
    carry = 0.0
    out = []
    for xi in x:
        carry = xi + rate * carry
        out.append(carry)
    return np.array(out, dtype=float)

def hill_saturation(x, alpha=100.0, gamma=1.5):
    x = np.asarray(x, dtype=float)
    return np.power(x, gamma) / (np.power(alpha, gamma) + np.power(x, gamma))

# Apply per channel
for ch in ['TV','Radio','Newspaper']:
    df[f'{ch}_adstock'] = adstock(df[ch].values, rate=0.5)
    df[f'{ch}_sat'] = hill_saturation(df[f'{ch}_adstock'].values, alpha=120.0, gamma=1.5)

df[['week','TV_sat','Radio_sat','Newspaper_sat']].head()


### Model: Ridge Regression with TimeSeries CV

In [ ]:

features = ['TV_sat','Radio_sat','Newspaper_sat','seasonality']
target = 'Sales'

X = df[features].values
y = df[target].values

tscv = TimeSeriesSplit(n_splits=5)
alphas = np.logspace(-4, 3, 50)

from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

ridge = Pipeline([
    ('scaler', StandardScaler(with_mean=True, with_std=True)),
    ('model', RidgeCV(alphas=alphas, cv=tscv))
])

ridge.fit(X, y)
alpha_ = ridge.named_steps['model'].alpha_
coef_ = ridge.named_steps['model'].coef_
intercept_ = ridge.named_steps['model'].intercept_

print("Selected alpha:", alpha_)
print("Intercept:", intercept_)
pd.Series(coef_, index=features).to_frame('coefficient')


### In-Sample Fit & Diagnostics

In [ ]:

df['pred'] = ridge.predict(X)

from sklearn.metrics import r2_score, mean_absolute_error
r2 = r2_score(y, df['pred'])
mae = mean_absolute_error(y, df['pred'])
print(f"R^2: {r2:.3f} | MAE: {mae:.3f}")

plt.plot(df['week'], y, label='Actual')
plt.plot(df['week'], df['pred'], label='Predicted')
plt.title('Actual vs Predicted Sales')
plt.xlabel('Week')
plt.ylabel('Sales')
plt.legend()
plt.show()


### Channel Contributions (Decomposition)

In [ ]:

coefs = pd.Series(ridge.named_steps['model'].coef_, index=features)

# Contribution via linear terms on transformed media
contri = pd.DataFrame(index=df.index)
for ch in ['TV_sat','Radio_sat','Newspaper_sat']:
    contri[ch] = coefs[ch] * df[ch+'_sat']

contri['seasonality'] = coefs['seasonality'] * df['seasonality']
contri['baseline'] = ridge.named_steps['model'].intercept_
contri['pred'] = contri.sum(axis=1)

contri_sum = contri[['TV_sat','Radio_sat','Newspaper_sat','seasonality']].sum()
share = (contri_sum / contri_sum.sum()).sort_values(ascending=False)
share.to_frame('contribution_share').style.format({'contribution_share': '{:.2%}'})


### Marginal ROI (mROI) at Current Spend Levels

In [ ]:

def dhill_dx(x, alpha=120.0, gamma=1.5):
    num = gamma * np.power(x, gamma-1) * np.power(alpha, gamma)
    den = np.power(alpha, gamma) + np.power(x, gamma)
    return num / (den**2)

def dadstock_dx(rate=0.5):
    return 1.0  # current period sensitivity

mroi = {}
for ch in ['TV','Radio','Newspaper']:
    x = df[f'{ch}_adstock'].values
    coef = coefs[f'{ch}_sat']
    cur_x = x[-1]
    d_sales_d_adstock = coef * dhill_dx(cur_x, alpha=120.0, gamma=1.5)
    d_sales_d_spend = d_sales_d_adstock * dadstock_dx(rate=0.5)
    mroi[ch] = float(d_sales_d_spend)

pd.Series(mroi).sort_values(ascending=False).to_frame('mROI (ΔSales per $1 spend)').round(6)


### Simple Budget Reallocation Heuristic

In [ ]:

extra_budget = 10.0
mroi_series = pd.Series(mroi).clip(lower=0)
weights = mroi_series / mroi_series.sum() if mroi_series.sum() > 0 else pd.Series({k: 1/3 for k in mroi_series.index})
allocation = (weights * extra_budget).round(2)
allocation.to_frame('recommended_extra_spend_$')



---

### Notes
- This is a compact, educational MMM; production models should include: holiday/price/promo controls, multiple saturation forms, channel interactions, priors (Bayesian MMM), and holdout validation (e.g., geo-experiments).  
- Always **triangulate** with experiments and directional business knowledge.

### References (non-link citations)
1. McCarthy & Perlich, *Modern Approaches to Media Mix Modeling*.  
2. Greene, *Econometric Analysis*.  
3. Rossi, Allenby & McCulloch, *Bayesian Statistics and Marketing*.  
